In [2]:
!pip install -U together datasets transformers tqdm


In [9]:
import sys
from pathlib import Path
import os
import json
import pandas as pd
import ast
from pathlib import Path
import yaml
from together.utils import check_file
from together import Together

# Find the real project root
current_dir = Path.cwd()
project_root = current_dir.parent  # because you're in "gen_fine_tuning", and project root is one level up

print("Project root:", project_root)

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.mult_lang import read_bio_json, read_bio_tsv



Project root: /Users/ofrihefetz/cross-lingual-idioms-1


In [10]:
LANGUAGE_CONFIG = {
    "turkish": {"format": "json", "source": "dodiom"},
    "japanese": {"format": "json", "source": "OpenMWE_sub"},
    "english": {"format": "tsv", "source": "id10m"},
    "french": {"format": "tsv", "source": "id10m"},
    "german": {"format": "tsv", "source": "id10m"},
    "italian": {"format": "tsv", "source": "id10m"},
    "spanish": {"format": "tsv", "source": "id10m"},
    "dutch": {"format": "tsv", "source": "id10m"},
    "chinese": {"format": "tsv", "source": "id10m"},
    "polish": {"format": "tsv", "source": "id10m"},
    "portuguese": {"format": "tsv", "source": "id10m"},
}

In [11]:
train_langs = ["turkish", "japanese", "english", "french", "german", "italian", "spanish", "dutch", "chinese", "polish", "portuguese"]
val_langs = ["english", "french", "german", "italian", "spanish", "dutch", "chinese", "polish", "portuguese"]

## saving the jsonl file in the right format

In [12]:
import os

# Automatically detect the project root (one level up from current script)
project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))

NameError: name '__file__' is not defined

In [13]:
TASK_CONFIG = {
    "dodiom": {
        "train_dir": os.path.join(project_root, "data/dodiom/{lang}/train.json"),
        "test_dir": os.path.join(project_root, "data/dodiom/{lang}/test.json"),
        "val_dir": None
    },
    "id10m": {
        "train_dir": os.path.join(project_root, "data/id10m/trainset/{lang}.tsv"),
        "test_dir": os.path.join(project_root, "data/id10m/testset/{lang}.tsv"),
        "val_dir": os.path.join(project_root, "data/id10m/devset/{lang}.tsv")
    },
    "open_mwe": {
        "train_dir": os.path.join(project_root, "data/OpenMWE_sub/train.json"),
        "test_dir": os.path.join(project_root, "data/OpenMWE_sub/test.json"),
        "val_dir": None
    },
    "magpie": {
        "train_dir": os.path.join(project_root, "data/magpie/MAGPIE_train_processed.jsonl"),
        "test_dir": os.path.join(project_root, "data/magpie/MAGPIE_test_processed_mini.jsonl"),
        "val_dir": None
    }
}


In [21]:

import random
import numpy as np
def get_data(lang: str, task: str) -> dict[str, pd.DataFrame]:
    """
    Load train, test, and validation data for a given language and task.
    Returns a dictionary: {'train': df, 'test': df, 'validation': df}
    """
    random.seed(42)
    np.random.seed(42)
    config = TASK_CONFIG.get(task)
    if config is None:
        raise ValueError(f"No task configuration found for task='{task}'.")
    
    if task == "open_mwe":
        assert lang == "japanese", f"❌ open_mwe only supports lang='japanese', but got lang='{lang}'"
    
    

    def load_json_lines(file_path: str) -> pd.DataFrame:
        if not os.path.exists(file_path):
            return pd.DataFrame()

        if task in {"open_mwe", "id10m"}:
            return read_bio_json(file_path)  # uses full JSON list structure
        elif task in {"magpie","dodiom"}:
            # Magpie is JSONL (one JSON per line)
            with open(file_path, "r", encoding="utf-8") as f:
                data = [json.loads(line) for line in f if line.strip()]
            return pd.DataFrame(data)
        else:
            raise ValueError(f"No JSON loader defined for task={task}")

    def load_tsv(file_path: str) -> pd.DataFrame:
        if not os.path.exists(file_path):
            return pd.DataFrame()
        return read_bio_tsv(file_path)  # Assuming this function exists

    def resolve_path(template: str | None) -> str | None:
        if template is None:
            return None
        return template.format(lang=lang) if "{lang}" in template else template

    def load_split(split: str) -> pd.DataFrame:
        path = resolve_path(config.get(f"{split}_dir"))
        if path is None or not os.path.exists(path):
            return pd.DataFrame()
        if path.endswith(".json") or path.endswith(".jsonl"):
            return load_json_lines(path)
        elif path.endswith(".tsv"):
            return load_tsv(path)
        else:
            raise ValueError(f"Unsupported file type: {path}")

    # Load all splits
    data = {
        "train": load_split("train"),
        "test": load_split("test"),
        "validation": load_split("val")
    }

    # Print summary
    for split, df in data.items():
        size = df.shape if not df.empty else "empty"
        print(f"✅ Loaded {split} for task='{task}', lang='{lang}': {size}")

    return data

In [22]:
import json

# SYSTEM_MESSAGE = """You are a professional linguist specializing in figurative language and your task is to analyse sentences that may contain an idiom, also known as an idiomatic expression. 
# This is a definition of idiom: 'A phrase, expression, or group of words that has a meaning different from the individual meanings of the words themselves, and employed to convey ideas in a non-literal or metaphorical manner'.
# Mark idioms only when their usage in the context is idiomatic/figurative and let literal meanings remain unmarked."""


# USER_PROMPT_TEMPLATE = """You are given one sentence in {language}, you are an expert of this language.
# If detected, write the idioms exactly as they are in the sentence, without any changes. Only answer in JSON.\n
# Sentence:{sentence}\n """

SYSTEM_MESSAGE = """You are a professional linguist specializing in figurative language and your task is to analyse sentences that may contain an idiom, also known as an idiomatic expression. 
This is a definition of idiom: 'A phrase, expression, or group of words that has a meaning different from the individual meanings of the words themselves, and employed to convey ideas in a non-literal or metaphorical manner'.
"""


USER_PROMPT_TEMPLATE = """You are given one sentence in {language}, you are an expert of this language.
Your task is to identify idioms **only if** they are used in an **idiomatic or figurative sense**. If the usage is literal, do not mark it.
- Output only the idioms that appear **exactly** as they are in the sentence, without any changes. Return the answer **in JSON format only**.\n
Sentence:{sentence}\n """

def row_to_chat_format(row):
    """
    Convert a single row of the dataset into Together AI conversational fine-tuning format.
    """
    language = row["language"]
    sentence = row["sentence"]
    idioms = row["true_idioms"]

    user_message = USER_PROMPT_TEMPLATE.format(language=language, sentence=sentence)
    assistant_response = json.dumps(idioms, ensure_ascii=False)

    return {
        "messages": [
            {"role": "system", "content": SYSTEM_MESSAGE},
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": assistant_response}
        ]
    }

In [ ]:
def get_json_file(language, task, data_type):
    df = get_data(language,task)
    seed = 42
    df = df["train"]
    with_idiom = df[df["true_idioms"].apply(lambda x: len(x) > 0)]
    without_idiom = df[df["true_idioms"].apply(lambda x: len(x) == 0)]
    
    if language == "turkish" and task == "dodiom":
            with_idiom = df[df["category"].apply(lambda x: x == "idiom")]
            without_idiom = df[df["category"].apply(lambda x: x == "nonidiom")]

    
    if data_type == "train":
        # Compute counts
        n_total = 1500
        percent_with_idiom = 0.27662410759249934
        n_with = round(n_total * percent_with_idiom)   # ≈216
        n_without = n_total - n_with 
    
    else: 
        n_total = 1 #150
        percent_with_idiom = 0 #0.19736842105263158
        n_with = round(n_total * percent_with_idiom)  
        n_without = n_total - n_with 
    
    # Sample
    sample_with_idiom = with_idiom.sample(n=n_with, random_state=42)
    sample_without_idiom = without_idiom.sample(n=n_without, random_state=42)
    
    if language == "turkish" and task == "dodiom":
        sample_without_idiom["true_idioms"] = sample_without_idiom["true_idioms"].apply(lambda _: [])

    
    # Combine and shuffle
    final_sample = pd.concat([sample_with_idiom, sample_without_idiom]).sample(frac=1, random_state=42).reset_index(drop=True)
    final_sample["language"] = language
    massage_df = final_sample.apply(row_to_chat_format, axis=1)

    # Create the output folder if it doesn’t exist
    output_folder = Path("jsonl_data")
    output_folder.mkdir(parents=True, exist_ok=True)

    # Build the full path
    output_path = output_folder / f"{language}_{n_total}_{seed}_{data_type}.jsonl"

    # Save the file
    massage_df.to_json(output_path, orient="records", lines=True)

    print(f"✅ Saved to {output_path}")

In [25]:
get_json_file("english", "id10m", "train")

✅ Loaded train for task='id10m', lang='english': (33757, 5)
✅ Loaded test for task='id10m', lang='english': (200, 5)
✅ Loaded validation for task='id10m', lang='english': (3724, 5)
✅ Saved to jsonl_data/english_1500_42_train.jsonl


## uplode the files to together AI 

In [7]:
# get the key to together API from keys.yaml 
current_dir = Path.cwd()
while current_dir != current_dir.parent:  # stop at root
    if (current_dir / "keys.yaml").exists():
        keys_file = current_dir / "keys.yaml"
        break
    current_dir = current_dir.parent
else:
    raise FileNotFoundError("Could not find keys.yaml in any parent directory")

# Load the keys
with open(keys_file, "r") as f:
    keys = yaml.safe_load(f)

TOGETHER_API_KEY = keys["TOGETHER_API_KEY"]

print(f"✅ Loaded API key from: {keys_file}")
client = Together(api_key=TOGETHER_API_KEY)


✅ Loaded API key from: /Users/ofrihefetz/cross-lingual-idioms-1/keys.yaml


In [26]:
# choose a file to check 
file = "/Users/ofrihefetz/cross-lingual-idioms-1/gen_fine_tuning/jsonl_data/english_1500_42_train.jsonl"
sft_report = check_file(file)
print(json.dumps(sft_report, indent=2))
assert sft_report["is_check_passed"] == True

# Upload the data to Together
train_file_resp = client.files.upload(file, check=True)
print(train_file_resp.id)  # Save this ID for starting your fine-tuning job

{
  "is_check_passed": true,
  "message": "Checks passed",
  "found": true,
  "file_size": 1562519,
  "utf8": true,
  "line_type": true,
  "text_field": true,
  "key_value": true,
  "has_min_samples": true,
  "num_samples": 1500,
  "load_json": true,
  "filetype": "jsonl"
}


Uploading file english_1500_42_train.jsonl: 100%|██████████| 1.56M/1.56M [00:01<00:00, 860kB/s]


file-468fa8c0-3d0a-451f-861e-c922d7f7b5fa


## Creating end point for FT modles 

In [ ]:
response = client.endpoints.create(
    display_name="Gemma-3-12B-Inference",
    model="kfirbar/gemma-3-12b-it-ADDTOEXPNAME-f399f126",  # Replace with your fine-tuned model
    hardware="2x_nvidia_h100_80gb_sxm",                    # ✅ confirmed hardware
    min_replicas= 1,
    max_replicas=1
)

print("✅ Endpoint created!")
print("ID:", response.id)
print("Status:", response.state)
print("Endpint name:", response.name)

✅ Endpoint created!
ID: endpoint-72738d88-6b49-4a7a-a345-1a25518709d1
Status: PENDING


In [22]:
response

DedicatedEndpoint(object='endpoint', id='endpoint-72738d88-6b49-4a7a-a345-1a25518709d1', name='kfirbar/gemma-3-12b-it-ADDTOEXPNAME-f399f126-15c6af46', model='kfirbar/gemma-3-12b-it-ADDTOEXPNAME-f399f126', type='dedicated', owner='kfirbar', state='PENDING', created_at=datetime.datetime(2025, 6, 27, 14, 37, 30, 356000, tzinfo=TzInfo(UTC)), display_name='Gemma-3-12B-Inference', hardware='2x_nvidia_h100_80gb_sxm', autoscaling=Autoscaling(min_replicas=1, max_replicas=1))

In [44]:
# status of the endpoint
client.endpoints.get(endpoint_id='endpoint-72738d88-6b49-4a7a-a345-1a25518709d1')

DedicatedEndpoint(object='endpoint', id='endpoint-72738d88-6b49-4a7a-a345-1a25518709d1', name='kfirbar/gemma-3-12b-it-ADDTOEXPNAME-f399f126-15c6af46', model='kfirbar/gemma-3-12b-it-ADDTOEXPNAME-f399f126', type='dedicated', owner='kfirbar', state='STOPPED', created_at=datetime.datetime(2025, 6, 27, 14, 37, 30, 356000, tzinfo=TzInfo(UTC)), display_name='Gemma-3-12B-Inference', hardware='2x_nvidia_h100_80gb_sxm', autoscaling=Autoscaling(min_replicas=1, max_replicas=1))

In [35]:
# start the endpoint
client.endpoints.update(endpoint_id='endpoint-72738d88-6b49-4a7a-a345-1a25518709d1', state='STARTED')

DedicatedEndpoint(object='endpoint', id='endpoint-72738d88-6b49-4a7a-a345-1a25518709d1', name='kfirbar/gemma-3-12b-it-ADDTOEXPNAME-f399f126-15c6af46', model='kfirbar/gemma-3-12b-it-ADDTOEXPNAME-f399f126', type='dedicated', owner='kfirbar', state='PENDING', created_at=datetime.datetime(2025, 6, 27, 14, 37, 30, 356000, tzinfo=TzInfo(UTC)), display_name='Gemma-3-12B-Inference', hardware='2x_nvidia_h100_80gb_sxm', autoscaling=Autoscaling(min_replicas=1, max_replicas=1))

In [42]:
# stop the endpoint
client.endpoints.update(endpoint_id='endpoint-72738d88-6b49-4a7a-a345-1a25518709d1', state='STOPPED')

DedicatedEndpoint(object='endpoint', id='endpoint-72738d88-6b49-4a7a-a345-1a25518709d1', name='kfirbar/gemma-3-12b-it-ADDTOEXPNAME-f399f126-15c6af46', model='kfirbar/gemma-3-12b-it-ADDTOEXPNAME-f399f126', type='dedicated', owner='kfirbar', state='STOPPING', created_at=datetime.datetime(2025, 6, 27, 14, 37, 30, 356000, tzinfo=TzInfo(UTC)), display_name='Gemma-3-12B-Inference', hardware='2x_nvidia_h100_80gb_sxm', autoscaling=Autoscaling(min_replicas=1, max_replicas=1))

## create end point for not the FT model! 

In [24]:
response = client.endpoints.create(
    display_name="kfirbar/Qwen/Qwen3-0.6B",
    model="Qwen/Qwen3-0.6B",  # Replace with your fine-tuned model
    hardware="1x_nvidia_h100_80gb_sxm",                    # ✅ confirmed hardware
    min_replicas= 1,
    max_replicas=1
)

print("✅ Endpoint created!")
print("ID:", response.id)
print("Status:", response.state)
print("Endpint name:", response.name)

✅ Endpoint created!
ID: endpoint-2a1507bb-bd80-44a5-9cce-6bec28797648
Status: PENDING
Endpint name: kfirbar/Qwen/Qwen3-0.6B-0abdf1c5


In [11]:
# start the endpoint
client.endpoints.update(endpoint_id='endpoint-b3a688d0-7bdf-4cd7-afde-c0b79569f1ae', state='STARTED')

DedicatedEndpoint(object='endpoint', id='endpoint-b3a688d0-7bdf-4cd7-afde-c0b79569f1ae', name='kfirbar/Qwen/Qwen3-4B-e24b8b92', model='Qwen/Qwen3-4B', type='dedicated', owner='kfirbar', state='PENDING', created_at=datetime.datetime(2025, 7, 9, 8, 35, 41, 850000, tzinfo=TzInfo(UTC)), display_name='kfirbar/Qwen/Qwen3-4B', hardware='1x_nvidia_h100_80gb_sxm', autoscaling=Autoscaling(min_replicas=1, max_replicas=1))

In [28]:
client.endpoints.get(endpoint_id='endpoint-2a1507bb-bd80-44a5-9cce-6bec28797648')

DedicatedEndpoint(object='endpoint', id='endpoint-2a1507bb-bd80-44a5-9cce-6bec28797648', name='kfirbar/Qwen/Qwen3-0.6B-0abdf1c5', model='Qwen/Qwen3-0.6B', type='dedicated', owner='kfirbar', state='STARTED', created_at=datetime.datetime(2025, 7, 9, 9, 6, 26, 634000, tzinfo=TzInfo(UTC)), display_name='kfirbar/Qwen/Qwen3-0.6B', hardware='1x_nvidia_h100_80gb_sxm', autoscaling=Autoscaling(min_replicas=1, max_replicas=1))

In [29]:
# stop the endpoint
client.endpoints.update(endpoint_id='endpoint-2a1507bb-bd80-44a5-9cce-6bec28797648', state='STOPPED')

DedicatedEndpoint(object='endpoint', id='endpoint-2a1507bb-bd80-44a5-9cce-6bec28797648', name='kfirbar/Qwen/Qwen3-0.6B-0abdf1c5', model='Qwen/Qwen3-0.6B', type='dedicated', owner='kfirbar', state='STOPPING', created_at=datetime.datetime(2025, 7, 9, 9, 6, 26, 634000, tzinfo=TzInfo(UTC)), display_name='kfirbar/Qwen/Qwen3-0.6B', hardware='1x_nvidia_h100_80gb_sxm', autoscaling=Autoscaling(min_replicas=1, max_replicas=1))

## change the parsing 

In [63]:
import json
import re
from pprint import pprint
from src.gen_utils import idioms_list_to_IOB  # Adjust import if needed

# === CONFIG ===
RESPONSES_PATH = "logs/gemma-3-12b-it-ADDTOEXPNAME-f399f126-15c6af46_english_on_english/responses.json"
NUM_EXAMPLES = 5

# === Load Responses ===
with open(RESPONSES_PATH, "r", encoding="utf-8-sig") as f:
    responses = json.load(f)

print(f"\n🔍 Loaded {len(responses)} responses. Showing first {NUM_EXAMPLES}:\n")

# === Process First N Examples ===
for i, example in enumerate(responses[:NUM_EXAMPLES]):
    print(f"\n====== Example {i + 1} ======\n")
    print(f"📝 Sentence: {example['sentence']}\n")

    raw_response = example["response"]
    print(f"🔹 Raw model output:\n{raw_response}\n")

    tokens = example["tokens"]
    gold_tags = example["labels"]

    # === Try to extract JSON and get idioms ===
    import re

    try:
        match = re.search(r"\{[\s\S]*?\}", raw_response)
        if match:
            parsed = json.loads(match.group())
            predicted_idioms = parsed.get("idioms", [])
            hallucinated = False
        else:
            print("⚠️ No valid JSON block found in response.")
            predicted_idioms = []
            hallucinated = True
    except Exception as e:
        print(f"❌ Failed to parse model response: {e}")
        predicted_idioms = []
        hallucinated = True

    print(f"✅ Parsed idioms: {predicted_idioms}\n")

    # === Convert to IOB ===
    predicted_tags = idioms_list_to_IOB(predicted_idioms, tokens, hallucinated)

    # === Display Side-by-Side Token Comparison ===
    print("🟦 Token-by-token comparison:")
    print("{:<15} {:<12} {:<12}".format("Token", "Gold", "Predicted"))
    print("-" * 45)
    for tok, gold, pred in zip(tokens, gold_tags, predicted_tags):
        print("{:<15} {:<12} {:<12}".format(tok.strip(), gold, pred))



🔍 Loaded 200 responses. Showing first 5:


====== Example 1 ======

📝 Sentence: A rock has broken the ice covering a lake.

🔹 Raw model output:
```json
{
  "idioms": []
}
```

✅ Parsed idioms: []

🟦 Token-by-token comparison:
Token           Gold         Predicted   
---------------------------------------------
A               O            O           
rock            O            O           
has             O            O           
broken          O            O           
the             O            O           
ice             O            O           
covering        O            O           
a               O            O           
lake            O            O           
.               O            O           

====== Example 2 ======

📝 Sentence: The ship broke the ice all the way.

🔹 Raw model output:
```json
{
  "idioms": [
    "broke the ice"
  ]
}
```

✅ Parsed idioms: ['broke the ice']

🟦 Token-by-token comparison:
Token           Gold         Predicted   
--------

In [53]:
answer = "```json\n{\n  \"idioms\": [\n    \"broke the ice\"\n  ]\n}\n```"
match = re.search(r"\{[\s\S]*?\}", answer)
if match:
    parsed = json.loads(match.group())
    predicted_idioms = parsed.get("idioms", [])

In [56]:
predicted_idioms

['broke the ice']

In [58]:
for i, example in enumerate(responses[:5]):
    print (f"\n====== Example {i + 1} ======\n")


====== Example 1 ======



In [163]:
import json
import re
import os
from src.gen_utils import idioms_list_to_IOB
from in_context_learning.src.id10m_utils import LABELS
from utils import calc_metrics_classification

# === Paths ===
RESPONSES_PATH = "/Users/ofrihefetz/cross-lingual-idioms-1/gen_fine_tuning/logs/old promt/Meta-Llama-3.1-70B-Instruct-Reference-ADDTOEXPNAME-90df6b6f_english_on_english/responses.json"
METRICS_PATH = os.path.join(os.path.dirname(RESPONSES_PATH), "metrics.json")

# === Load file ===
with open(RESPONSES_PATH, "r", encoding="utf-8-sig") as f:
    responses = json.load(f)

updated_responses = []
all_gold = []
all_pred = []

# === Fix each example ===
for ex in responses:
    tokens = ex["tokens"]
    labels = ex["labels"]
    response_text = ex["response"]

    try:
        # Fix markdown JSON block
        match = re.search(r"\{[\s\S]*?\}", response_text)
        if match:
            parsed = json.loads(match.group())
            predicted_idioms = parsed.get("idioms", [])
            hallucinated = False
        else:
            predicted_idioms = []
            hallucinated = True
    except Exception as e:
        print(f"⚠️ JSON parsing failed for: {ex['sentence']}")
        predicted_idioms = []
        hallucinated = True

    predicted_tags = idioms_list_to_IOB(predicted_idioms, tokens, hallucinated)

    ex["predicted_idioms"] = predicted_idioms
    ex["predicted_tags"] = predicted_tags

    updated_responses.append(ex)
    all_gold.extend(labels)
    all_pred.extend(predicted_tags)

# === Save corrected responses ===
with open(RESPONSES_PATH, "w", encoding="utf-8-sig") as f:
    json.dump(updated_responses, f, indent=2, ensure_ascii=False)
print(f"✅ Fixed responses saved to: {RESPONSES_PATH}")

# === Compute new metrics ===
metrics, _, _ = calc_metrics_classification(all_gold, all_pred, labels=LABELS)

with open(METRICS_PATH, "w", encoding="utf-8-sig") as f:
    json.dump(metrics, f, indent=2, ensure_ascii=False)
print(f"✅ New metrics saved to: {METRICS_PATH}")
print(f"\n📊 Updated metrics:\n{metrics}")


✅ Fixed responses saved to: /Users/ofrihefetz/cross-lingual-idioms-1/gen_fine_tuning/logs/old promt/Meta-Llama-3.1-70B-Instruct-Reference-ADDTOEXPNAME-90df6b6f_english_on_english/responses.json
✅ New metrics saved to: /Users/ofrihefetz/cross-lingual-idioms-1/gen_fine_tuning/logs/old promt/Meta-Llama-3.1-70B-Instruct-Reference-ADDTOEXPNAME-90df6b6f_english_on_english/metrics.json

📊 Updated metrics:
{'accuracy': 0.9263766352296927, 'precision': 0.7854693431126551, 'recall': 0.8579599496181297, 'f1': 0.8173350750606367}


In [6]:
client.fine_tuning.download(id="ft_resp.id")

KeyboardInterrupt: 